Lista 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

In [9]:
alphas = [0.01, 0.05]
sigmas = [0.01, 0.5, 1]
ns = [100, 1000, 5000]

In [15]:
def pu_b1(sigma, alpha, x):
    return stats.norm.ppf(1-alpha/2) * sigma / np.sqrt(np.sum(x-np.mean(x))**2)

In [16]:
def pu_b0(sigma, alpha, x):
    return stats.norm.ppf(1 - alpha/2) * sigma * np.sqrt((1/ len(x) + np.mean(x)**2 / np.sum(x-np.mean(x))**2))

In [17]:
def b1_estim(x,y):
    return np.sum((x - np.mean(x)) * (y))/np.sum((x-np.mean(x))**2)

In [18]:
def b0_estim(x,y):
    return np.mean(y) - b1_estim(x,y) * np.mean(x)

In [19]:
def generate_sample(n, b0, b1, sigma):
    eps = np.random.normal(0, sigma, n)
    x = np.linspace(1, n+1, n)
    y = b1 * x + b0 + eps
    return x, y

In [ ]:
def mc_test(n, sigma, b0, b1, mc, alpha):
    b1_in, b0_in = 0,0
    for _ in range(mc):
        x, y = generate_sample(n, b0, b1, sigma)
        b1_hat, b0_hat = b1_estim(x,y), b0_estim(x, y)
        pu_b0_, pu_b1_ = pu_b0(sigma, alpha, x), pu_b1(sigma, alpha, x)
        if b1_hat - pu_b1_ < b1 < b1_hat + pu_b1_:
            b1_in += 1
        if b0_hat - pu_b0_ < b0 < b0_hat + pu_b0_:
            b0_in += 1
    return b1_in / mc, b0_in / mc

In [29]:
print(mc_test(1000, 0.01, 1, 2, 1000, 0.5))

(1.0, 1.0)
